In [1]:
%matplotlib inline

import os
import os.path as osp
import sys
# import Image
# import sys
import argparse
import os, tempfile, glob, shutil
import random
from datetime import datetime
random.seed(datetime.now())

# import gaussian_process
import GPy
import GPyOpt
import subprocess as sub
import numpy as np
import scipy.spatial
import math
import matplotlib.pyplot as plt
from numpy import linalg as LA

from scipy import optimize
import time

global g_blender_executable_path,  blank_file, render_code, model_file, table_file, temp_dirname, epsilon, \
       filename, time, location_gt, rotation_gt


epsilon = 1e-6


if os.environ.get('BLENDER_PATH') == None:
    print("Please set BLENDER_PATH in bashrc!")
    sys.exit()

g_blender_executable_path = os.environ['BLENDER_PATH']

obj_model_folder = 'obj_models/'
table_model_folder = 'table_models/'

g_blank_blend_file_path = 'blank.blend'
syn_images_folder = 'rendered_images'
if not os.path.exists(syn_images_folder):
    os.mkdir(syn_images_folder)

all_models = []
for file in os.listdir(obj_model_folder):
    if file.endswith(".obj"):
        all_models.append(file)

print(all_models)

# random select model in all_models
# To Do

# model_name and table_name are hard coded
model_name = 'expo_dry_erase_board_eraser'
robot_name = 'reflex'

########################### Input and GT ####################
fps = 24

path = '/home/zhusj/Dropbox/16fall/learningPhysics/data/data/trial4/'

filelist = ['exp1', 'exp2', 'exp3']
timelist = [8,4,1]

location_gt = []
rotation_gt = []

for file in filelist:
    count = 0
    filename = path+file+'.txt'
    with open(filename) as f:

        obj_start_config = []
        hand_start_config = []
        obj_end_config = []
        hand_end_config = []
        for line in f: # read rest of lines
            if count == 7: 
                obj_end_config.append([float(x) for x in line.split()])
            count += 1

    print obj_end_config
    myorder=[0,1,2,6,3,4,5]
    obj_end_config = [ obj_end_config[0][i] for i in myorder]
    print obj_end_config


    obj_end_config = np.array(obj_end_config)
    print obj_end_config.shape

    location_gt.append(obj_end_config[:3])
    rotation_gt.append(obj_end_config[3:])

print location_gt, rotation_gt
##############################################

table_name = 'table'
arm_name ='arm'
image_name = 'demo_img.png'
output_img = osp.join(syn_images_folder, image_name)
output_blend = osp.join(syn_images_folder, image_name+'.blend')

# call blender to render images
blank_file = osp.join(g_blank_blend_file_path)
render_code = osp.join('render_physics_reflex_expo.py')
model_file = osp.join(obj_model_folder, model_name+ '.obj')
robot_file = osp.join(obj_model_folder, robot_name+ '.obj')
table_file = osp.join(table_model_folder, table_name+ '.obj')
# print render_code
# print model_file
temp_dirname = '/home/zhusj/repos/blender_scene_generator/test'  #tempfile.mkdtemp()

/home/zhusj/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


['dasani_water_bottle.obj', 'expo_dry_erase_board_eraser.obj', 'reflex.obj', 'pliers.obj', 'folgers_classic_roast_coffee.obj', 'expo_dry_erase_board_eraser_00000.obj', 'reflex_nc.obj', 'crayola_24_ct.obj', 'robotiq.obj', 'scotch_duct_tape.obj', 'folgers_classic_roast_coffee_00000.obj', 'kleenex_tissue_box.obj']
[[0.543561, -0.123172, 0.54746, -0.00114687, 0.00659373, -0.781483, 0.62389]]
[0.543561, -0.123172, 0.54746, 0.62389, -0.00114687, 0.00659373, -0.781483]
(7,)
[[0.532153, -0.0257914, 0.548445, -0.00623893, -0.00737948, 0.902664, -0.430238]]
[0.532153, -0.0257914, 0.548445, -0.430238, -0.00623893, -0.00737948, 0.902664]
(7,)
[[0.48143, -0.0221645, 0.54965, -0.00235522, -0.00839558, 0.972091, -0.234441]]
[0.48143, -0.0221645, 0.54965, -0.234441, -0.00235522, -0.00839558, 0.972091]
(7,)
[array([ 0.543561, -0.123172,  0.54746 ]), array([ 0.532153 , -0.0257914,  0.548445 ]), array([ 0.48143  , -0.0221645,  0.54965  ])] [array([ 0.62389   , -0.00114687,  0.00659373, -0.781483  ]), arr

In [2]:
def run_physics_engine(filename, mass, friction, force, time):

    location =[]
    rotation = []    

    render_cmd = '%s %s -b --python %s -- %s %s %s %s %s %s %s %s %s' % \
    (g_blender_executable_path, blank_file, render_code, robot_file, model_file, time, \
    filename, force_gt, mass, friction, table_file, temp_dirname)

    p = os.popen(render_cmd,"r")
    while 1:
        line = p.readline()
        if not line: break
        if "location" in line: 
            print "output:", line
            vals = line.split(',')
            count = 0
            for val in vals:
                count += 1
                if count == 1:
                    x= float(val[-6:])
                if count == 2:
                    y= float(val[-7:])

        if "rotation" in line: 
            print "output:", line
            vals = line.split(',')
            count = 0
            for val in vals:
                count += 1
                if count == 1:
                    qw= float(val[-6:])
                if count == 2:
                    qx= float(val[-6:])
                if count == 3:
                    qy= float(val[-6:])
                if count == 4:
                    newstr = val[-10:-3].replace("=", "")
                    qz= float(newstr)
                    
    location.append([x, y])
    location = np.asarray(location)
    rotation.append([qw,qx,qy,qz])
    rotation = np.asarray(rotation)
    return location, rotation

In [4]:
def myf(mass,friction):

#     mass = X[0][0]
#     friction = X[0][1]
    
    y_new = 0
    count = 0
    for file in filelist:        
        filename = path+file+'.txt'
        time = timelist[count]*fps
#         print 'input: ', filename, mass, friction, force_gt, time
        
        location, rotation = run_physics_engine(filename, mass, friction, force_gt, time)  
        dist = np.sqrt(np.sum(((location-location_gt[count][0:2])**2)))
        q_dist = 1 - np.sum(np.multiply(rotation, rotation_gt[count]))**2
        

        print dist, q_dist
        y_new += dist + q_dist
        count += 1

    print "X, y_new",mass,friction,y_new

    return y_new

In [5]:
# RS_result =[]
# RS_time =[]
force_gt =48
for count in xrange(1,5):
    random.seed(datetime.now())
    start_time = time.time()
    best_y = 10000
    best_f = 1
    best_m = 1
    result = []
    elapsed_time = 0
    ################## Grid Search #############
    while elapsed_time < 1200:
        m = 0.5*random.random()
        f = 0.7*random.random()

        y = myf(m,f)
        if y < best_y:
            best_y = y
            best_m = m
            best_f = f
            
        result.append([m,f,y,best_m,best_f,best_y])
        elapsed_time = time.time() - start_time
    print "optima!!!!!!!!!!!!!!!!: ", best_m,best_f,best_y
    RS_result.append(result)


output: location:  <Vector (0.5596, -0.1398, 0.7755)>

output: rotation:  <Quaternion (w=0.5320, x=-0.0097, y=-0.0068, z=-0.8466)>

0.0231028116254 0.0128649538759
output: location:  <Vector (0.5628, -0.0540, 0.7749)>

output: rotation:  <Quaternion (w=0.3390, x=0.0010, y=0.0010, z=-0.9408)>

0.0416528957332 0.00979471448926
output: location:  <Vector (0.4811, 0.0075, 0.7749)>

output: rotation:  <Quaternion (w=0.4271, x=0.0024, y=0.0023, z=-0.9042)>

0.0296663354705 0.0413252090385
X, y_new 0.433906991962 0.561599629424 0.158406920233
output: location:  <Vector (0.5548, -0.1366, 0.7746)>

output: rotation:  <Quaternion (w=0.5053, x=0.0053, y=0.0035, z=-0.8629)>

0.0175107482707 0.0206714593907
output: location:  <Vector (0.5612, -0.0542, 0.7749)>

output: rotation:  <Quaternion (w=0.3156, x=0.0009, y=0.0008, z=-0.9489)>

0.0406297521893 0.0152762074174
output: location:  <Vector (0.4733, 0.0010, 0.7747)>

output: rotation:  <Quaternion (w=0.3411, x=0.0135, y=0.0068, z=-0.9399)>

0.024

In [13]:
data = np.empty([4,30])
for i in xrange(0,4):    
    print len(RS_result[i])
    for j in xrange(0,len(RS_result[i])):
        data[i][j] = RS_result[i][j][5]

24
25
26
26


In [14]:
np.mean(data, axis=0)

array([  1.21352544e-001,   1.11080016e-001,   1.04895638e-001,
         1.04895638e-001,   1.01233660e-001,   1.01233660e-001,
         9.05312212e-002,   9.05312212e-002,   8.86737116e-002,
         8.86737116e-002,   8.53842505e-002,   8.53842505e-002,
         8.53842505e-002,   8.53842505e-002,   8.53842505e-002,
         8.53842505e-002,   8.53842505e-002,   8.53842505e-002,
         8.53842505e-002,   8.53842505e-002,   8.53842505e-002,
         8.53842505e-002,   8.53842505e-002,   8.01975749e-002,
         5.96461618e-002,   4.36556422e-002,   2.17504569e-313,
         6.68476846e-313,   1.73011718e-310,   1.72975813e-310])

In [15]:
np.std(data, axis=0)

array([ 0.02330244,  0.0123582 ,  0.01696089,  0.01696089,  0.01366249,
        0.01366249,  0.00904391,  0.00904391,  0.00717711,  0.00717711,
        0.00632147,  0.00632147,  0.00632147,  0.00632147,  0.00632147,
        0.00632147,  0.00632147,  0.00632147,  0.00632147,  0.00632147,
        0.00632147,  0.00632147,  0.00632147,  0.01129923,  0.03622453,
        0.04405995,  0.        ,  0.        ,  0.        ,  0.        ])